# Performance simulation in live clinical workflow

### 1. Import required libraries

In [ ]:
from mlpipeline.evaluation import model_evaluater
from mlpipeline import mlpipeline_settings
from datetime import timedelta
from scipy.stats import norm
from math import sqrt
from matplotlib.dates import DateFormatter
import pandas as pd
import math
import numpy as np
import matplotlib.pyplot as plt
import re

### 2. Provide some general information about the log

In [ ]:
# General inputs
use_case = 'del'
customer = 'HDZ'

date_logs =['20210219', '20210226', '20210305','20210319','20210409','20210416','20210507','20210521','20210528','20210604','20210611','20210618','20210625','20210709','20210723','20210806']

path = r'logs'
path_log_file = f'{path}/{customer}/{date_logs[-1]}'
dates_indicator = '_'.join(date_logs)

# If the last of item of date_logs is earlier than the available outcome, manually update the path below
path_outcome_file = f'{path_log_file}/{date_logs[-1]}_{customer.lower()}_{use_case[0:3]}.txt'

### 3. Read in the log file and create a dataframe containing all observation in the log

In [ ]:
### STEP 0: merge the log files
aggregated_log=[]

for date_log in date_logs:
    with open(f'{path}/{customer}/{date_log}/{customer}_{date_log}-{use_case[0:3]}-model.log', "r") as log:
        for line in log:
            aggregated_log.append(line)


### 4. Extract the predictions from the RESPONSE line in the log

In [ ]:
prediction_data = pd.DataFrame(columns=['CASEID','DATE', 'OBS','BELIEF'])
cases=[]
beliefs = []
observations = []
dates=[]
for line in aggregated_log:
    if re.search('RESPONSE',line):
        step1 = re.sub('^.* RESPONSE ','',line)
        step2 = re.sub('\\\\n".*$','',step1)
        step3 = re.sub('^.*:"','',line)[:19]
        
        step5 = line.replace("\\","")
        obs = re.sub('}n',': ',step5).split('": "')[2]
        result = re.sub('\n', '', step2)
        case = result.split(': ')[0]
        belief = result.split(': ')[2].split(',')[0]
        cases.append(case)
        # NOTE: in 3.0 score != belief, BELIEF = EXP(SCORE)

        if ('DELIRIUM 1' in line )| ('SEPSIS 1' in line )|('AKI 1' in line ):
            beliefs.append(math.exp(float(belief)))
        else:
            beliefs.append(1-math.exp(float(belief)))        
        
        dates.append(step3)
        observations.append(obs)
            
prediction_data.CASEID = cases
prediction_data.BELIEF = beliefs  
prediction_data.DATE = dates
prediction_data.OBS = observations
prediction_data = prediction_data.sort_values(['CASEID','DATE'])

# Possibility to save intermediary steps 
prediction_data.to_csv(f'{path_log_file}/{use_case}/prediction_data_{dates_indicator}_{use_case[0:3]}.csv', index = False, sep = ";")

In [ ]:
prediction_data = prediction_data.drop_duplicates()
prediction_data['CASEID'] = prediction_data['CASEID'].astype(int)
prediction_data.head()

prediction_data['DATETIME'] = pd.to_datetime(prediction_data['DATE'], format="%Y-%m-%dT%H:%M:%S")

# date is only on day level
prediction_data['DATE'] = prediction_data['DATETIME'].apply(lambda x: x.date())

# for predicitons that are belonging to a same medical case and that has same observations, only take the last instance.
prediction_data['DATETIME'] = prediction_data.groupby(['CASEID','DATE','OBS','BELIEF'])['DATETIME'].transform(max)

### 5. Select a specific medical case

In [ ]:
medical_case = prediction_data[prediction_data.CASEID == 1683205]

### 6. Calculate predictions with models trained in different hospitals

In [ ]:
# models for delirium
hf_datetime = pd.to_datetime('2021-04-16 07:57:47', format="%Y-%m-%d %H:%M:%S") # when the model have been changed
model_v_old_HDZ = 1611085421
model_v_new_HDZ = 1618005122
models = [model_v_old_HDZ, model_v_new_HDZ, model_v_old_MHS, model_v_new_MHS, model_v_old_MKN, model_v_new_MKN]    

In [ ]:
def rescale_belief(o, belief):
    b = math.exp(belief)
    # belief in DISEASE 0 is 1 - belief in DISEASE 1
    if o == 0:
        b = 1 - b
  
  # transform the belief thresholds to risk score thresholds
    rescaled = np.interp(b, belief_thresholds, risk_score_thresholds)
    return rescaled

belief_thresholds = [0, 0.2, 0.4, 1]
risk_score_thresholds = [0, 0.5, 0.75, 1]

def generate_observations(obs_df):
    for i in range(len(obs_df)):
        yield {
            "inputs": obs_df.OBS_COMPLETE.iloc[i],
            "label": "",
            "caseid": obs_df.CASEID.iloc[i],
            "med_day": "",
            "hour_group": ""
        }
        
def get_confidence_interval(p, n, confidence = 0.95):
    if n < 1:
        return [float('NaN'), float('NaN')]
    left_bound = 0.5 - confidence / 2
    right_bound = 0.5 + confidence / 2
    z_left = norm.ppf(left_bound)
    z_right = norm.ppf(right_bound)
    return [round(p + z_left * sqrt(p * (1 - p) / n),3),
            round(p + z_right * sqrt(p * (1 - p) / n),3)]


In [ ]:
# Calculate predictions with the delirium model of each hospital
use_case_complete_name = 'delirium'

prediction_data_old = model_evaluater.run(
    generate_observations(medical_case_complete), use_case_complete_name , mlpipeline_settings, model_version=models[0])

prediction_data = model_evaluater.run(
    generate_observations(medical_case_complete), use_case_complete_name , mlpipeline_settings, model_version=models[1])

medical_case_complete['BELIEF_1'] = prediction_data_old.apply(lambda row: rescale_belief(row['PREDICTION'],row['SCORE']),axis=1)
medical_case_complete['BELIEF_2'] = prediction_data.apply(lambda row: rescale_belief(row['PREDICTION'],row['SCORE']),axis=1)
medical_case_complete['ADJUSTED_BELIEF'] = np.where(pd.to_datetime(medical_case_complete.DATETIME, format="%Y-%m-%d %H:%M:%S")<hf_datetime, medical_case_complete['BELIEF_1'], medical_case_complete['BELIEF_2'])

In [ ]:
# Plot the performance for the differents hospital in the same figure to be compared 
caseid = medical_case_complete.CASEID.unique()
op_day = pd.to_datetime('2021/07/13', format="%Y/%m/%d")
model_change_day = pd.to_datetime('2021-04-16 07:57:47', format="%Y-%m-%d %H:%M:%S")
case_obs_complete = medical_case_complete
myDates = case_obs_complete['DATETIME']
myDates = pd.to_datetime(case_obs_complete['DATETIME'], format="%Y-%m-%d %H:%M:%S")
myValues_comp = case_obs_complete['ADJUSTED_BELIEFHDZ'].to_numpy()

fig, axs = plt.subplots(1, figsize=(30,15))
fig.set_facecolor('w')
myFmt = DateFormatter('%m-%d')

axs.plot(myDates,case_obs_complete['ADJUSTED_BELIEF'].to_numpy(), drawstyle="steps-post")
axs.xaxis.set_major_formatter(myFmt)
axs.set_xlabel('Days', size=24)
axs.set_ylabel('Belief', size=24)
axs.axhspan(0.0,0.5, facecolor='g', alpha= 0.12)
axs.axhspan(0.5,0.75, facecolor='orange', alpha= 0.12)
axs.axhspan(0.75,1, facecolor='r', alpha= 0.12)
axs.axvline(x=op_day,label='Operation day '+str(op_day.date()), c='r')
axs.set_ylim(ymin=0, ymax = 1.01)
axs.tick_params(axis='both', which='major', labelsize=20)

if myDates.min()< model_change_day<myDates.max() :
    axs.axvline(x=model_change_day,label='Model change', c='g')

axs.legend(prop={'size': 20})

plt.figure()
fig.savefig(f'{path}/medical_case_'+str(int(caseid))+'.jpg', bbox_inches = 'tight')